Script 1:
Including vcf data into genbank files:

In [ ]:
from Bio import SeqIO,Seq
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqFeature
import vcf

def vcftogenbank(reference, sample):
    
    genbank=SeqIO.read(reference,"genbank") #open reference genbank file
    new_features=[]

    vcf_file=vcf.Reader(filename="vcf_files/"+sample+"_c.vcf.gz",compressed="True") #open vcf file
    new_sequence=""
    new_sequence=str(genbank.seq.strip())
    old=len(new_sequence)


    dif=0

    for SNP in vcf_file:
        new_sequence=new_sequence[:SNP.POS-1+dif]+str(SNP.ALT[0]).lower()+new_sequence[SNP.POS+len(SNP.REF)-1+dif:] #including sequence changes into new sequence
        difdif=0
        difdif=len(SNP.ALT[0])-len(SNP.REF) #in case of INDELs sequence positions need to be adapted
        dif=dif+difdif

        for feature in genbank.features: #updating features
            if (feature.location.start.position > SNP.POS) and (feature.location.end.position > SNP.POS): #if SNP upstream of feature      
                feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start+difdif),
                                                                     SeqFeature.ExactPosition(
                                                                    feature.location.end+difdif),
                                                                        feature.location.strand)
            elif (feature.location.start.position <= SNP.POS) and (feature.location.end.position > SNP.POS): #if SNP is within feature
                feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start),
                                                                     SeqFeature.ExactPosition(
                                                                    feature.location.end+difdif),
                                                                    feature.location.strand)
                if "translation" in feature.qualifiers:
                    transcript=""
                    translation=""
                    transcript=transcript+new_sequence[feature.location.start-1:feature.location.end] 
                    transcript=Seq(transcript) #getting feature transcript

                    if feature.location.strand == -1:
                        transcript=transcript.reverse_complement()

                    translation=transcript.translate() #translate trancript and including in feature description
                    feature.qualifiers["translation"]=str(translation)


            elif (feature.location.start.position < SNP.POS) and (feature.location.end.position >= SNP.POS): #if SNP is downstream of feature
                feature.location = SeqFeature.FeatureLocation(SeqFeature.ExactPosition(feature.location.start),
                                                                     SeqFeature.ExactPosition(
                                                                    feature.location.end),
                                                                        feature.location.strand)



    for feature in genbank.features: #getting new features
        new_features.append(feature)

    new_seq=Seq(new_sequence) #getting new sequence

    genbank.seq=new_seq
    genbank.features=new_features
    
    
    start=0
    end=0
    for feature in genbank.features:
        if feature.type =="repeat_region":
            if "TRL" in feature.qualifiers["note"][0]:
                start=int(feature.location.end.position)
            elif "TRS" in feature.qualifiers["note"][0]:
                end=int(feature.location.start.position)
                
    fastaseq=genbank.seq[start-1:end] #truncating sequence by excluding terminal repeats
             
                
    

    outfile=open(sample+".gb","w")

    SeqIO.write(genbank,outfile,"genbank") #writing new genbankfile
    
    outfile.close()
    return str(fastaseq) #return truncated genome sequence




Script 2:
Analysing vcf data for further usage:

In [ ]:
import csv
import vcf
import os
from matplotlib import pyplot as plt
import numpy as np
from Bio import SeqIO,Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

nucl=("A","T","C","G") #DNA bases

out_SNPs="sample;REF;POS;ALT;changetype;gene;aachange;AF;DP;dS_or_dN\n" #SNP table header
out_gene_freq="sample"

record=SeqIO.read("HSV-1_F-BAC.gb","genbank") #annotated reference genome

gene_list={}
gene_syn={}
gene_nons={}

for feature in record.features:
    if feature.type=="CDS":
        if "R" not in feature.qualifiers["gene"][0]:
            gene_list[feature.qualifiers["gene"][0]]=0
            gene_syn[feature.qualifiers["gene"][0]]=0
            gene_nons[feature.qualifiers["gene"][0]]=0
            out_gene_freq=out_gene_freq+";"+feature.qualifiers["gene"][0]
            
        
out_gene_freq=out_gene_freq+"\n" #gene frequency table header
out_dNdS=out_gene_freq
out_EGFP="sample;EGFP frequency\n"

csv_file=csv.reader(open("samples_min50DP.csv"), delimiter=";") #table with samples and where to find them
samples_folder=[]

for sample in csv_file: #loading samplelist
    samples_folder.append(sample)

SNPs=[] #future SNP list
gene_freq=[] #future gene frequency list
EGFP_freq=[] #future EGFP frequency list
namelist=[] #future sample list, replacing Rep1 samplenames
dNdS=[]
for sample in samples_folder:
    
    vcf_file=vcf.Reader(filename=sample[1]+"/vcf_files/"+sample[0]+"_lofreq.vcf.gz",compressed="True") #loading vcf specified by sample and folder
    name=""
    if "Rep" not in sample[0]: #rebranding Rep1 samples because they are not stated as such
        name=sample[0]+"Rep1"
    else:
        name=sample[0]
        
    namelist.append(name)
    
    for SNP in vcf_file: #loading SNPs
        for ALT in SNP.ALT: #for lofreq calls not necessary but just in case there are multiple variants per nucleotide site
            
            if SNP.INFO["AF"]>=0.05: #only using SNPs with allele frequencies higher than 5%
                change="NONC"
                for feature in record.features:                
                    if feature.type == "CDS": 
                        if feature.location.start < SNP.POS and feature.location.end >= SNP.POS: #checking for mutations within coding sequences
                            trans_old=""
                            trans_new=""
                            change="SYN"
                            for segment in feature.location.parts: #in case of multiple parts
                                trans_old=trans_old+record.seq[segment.start:segment.end]
                                if segment.start<SNP.POS and segment.end>=SNP.POS:
                                    trans_new=trans_new+record.seq[segment.start:SNP.POS-1]+str(ALT)+record.seq[SNP.POS:segment.end]
                                else:
                                    trans_new=trans_new+record.seq[segment.start:segment.end]
                            if feature.location.strand==-1:
                                trans_old=trans_old.reverse_complement()
                                trans_new=trans_new.reverse_complement()
                            trans_old=trans_old.translate()
                            trans_new=trans_new.translate()
                            gene=feature.qualifiers["gene"][0] #note: check how nametags are qulified in your genbank file!
                            if trans_old!=trans_new:
                                i=1
                                c=0
                                for old,new in zip(trans_old,trans_new):
                                    if old!=new:
                                        change=old+str(i)+new

                                        c=c+1
                                    i=i+1
                                if c>1:
                                    print("fail",feature.qualifiers['gene'][0]) #something went wrong if there are more than one changes per SNP per CDS e.g. INDELs
                            syn=0
                            non=0
                            for base in nucl: #checking for SYN and NONS sites for this SNP
                                mRNA=""
                                trans_old=""
                                if base!=SNP.REF:
                                    for segment in feature.location.parts:
                                        trans_old=trans_old+record.seq[segment.start:segment.end]
                                        if segment.start<SNP.POS and segment.end>=SNP.POS:
                                            mRNA=mRNA+record.seq[segment.start:SNP.POS-1]+base+record.seq[SNP.POS:segment.end]
                                        else:
                                            mRNA=mRNA+record.seq[segment.start:segment.end]
                                    if feature.location.strand==-1:
                                        mRNA=mRNA.reverse_complement()
                                        trans_old=trans_old.reverse_complement()
                                    prot=mRNA.translate()
                                    trans_old=trans_old.translate()
                                    if prot==trans_old:
                                        syn+=1
                                    else:
                                        non+=1




                if change=="NONC": #line in case of non coding SNPs
                    line=[name,SNP.REF,SNP.POS,ALT,change,"-","-",SNP.INFO["AF"],SNP.INFO["DP"],"-"]
                elif change=="SYN": #line in case of synonymous SNPs
                    line=[name,SNP.REF,SNP.POS,ALT,change,gene,"-",SNP.INFO["AF"],SNP.INFO["DP"],str(SNP.INFO["AF"]/syn)]
                else: #line in case of non synonymous SNPs
                    line=[name,SNP.REF,SNP.POS,ALT,"NONS",gene,change,SNP.INFO["AF"],SNP.INFO["DP"],str(SNP.INFO["AF"]/non)]

                SNPs.append(line)
                
              #calculating gene frequencies from per site depth data  
    cov=[]            
    for line in (open(sample[1]+"/coverage/"+sample[0]+"_cov_av.txt")):
        cov.append(float(line.split()[2]))
    av=0.0
    for site in cov:
        av+=site
    av=av/len(cov)
    for feature in record.features:
        if feature.type=="CDS":
            gene_cov=cov[feature.location.start-1:feature.location.end]
            gene_av=0.0
            for site in gene_cov:
                gene_av+=site
            gene_av=gene_av/len(gene_cov)
            gene_av=gene_av/av
            if "R" not in feature.qualifiers["gene"][0]:
                gene_list[feature.qualifiers["gene"][0]]=gene_av

    line=[name]
    for gene in gene_list:
        line.append(gene_list[gene])
    gene_freq.append(line)

    
    
    
    for a, e in zip(csv.reader(open(sample[1]+"/coverage/depth_av.csv"),delimiter=";"),csv.reader(open(sample[1]+"/coverage/depth_EGFP.csv"),delimiter=";")):
        if (a[0] and e[0]) == sample[0]:
            EGFP_freq.append((name,float(e[1])/float(a[1])))
            

for sample in namelist:
    line=[sample]
    for SNP in SNPs:
        if (SNP[0]==sample) and (SNP[4]!="NONC") and (SNP[2]!=64393) and (SNP[2]!=64394) and (SNP[2]!=64395) and ("R" not in SNP[5]):
            if SNP[4]=="SYN":
                gene_syn[SNP[5]]+=float(SNP[9])
            elif SNP[4]=="NONS":
                gene_nons[SNP[5]]+=float(SNP[9])
    for gene in gene_list:
        if gene_syn[gene]==0.0:
            gene_nons[gene]=gene_nons[gene]/(0.01/3)
        else:
            gene_nons[gene]=gene_nons[gene]/gene_syn[gene]
            
        line.append(gene_nons[gene])
        gene_syn[gene]=0
        gene_nons[gene]=0
    dNdS.append(line)
    
for d,f,e in zip(dNdS,gene_freq,EGFP_freq):
    line_d=""
    line_f=""
    line_e=""
    
    for el in d:
        line_d=line_d+str(el)+";"
    line_d=line_d[:-1]+"\n"
    
    for el in f:
        line_f=line_f+str(el)+";"
    line_f=line_f[:-1]+"\n"
    
    for el in e:
        line_e=line_e+str(el)+";"
    line_e=line_e[:-1]+"\n"
    
    out_dNdS=out_dNdS+line_d
    out_gene_freq=out_gene_freq+line_f
    out_EGFP=out_EGFP+line_e
    
file_dNdS=open("dNdS.csv","w")
file_gene_freq=open("gene_freq.csv","w")
file_EGFP=open("EGFP.csv","w")

file_dNdS.write(out_dNdS[:-1])
file_gene_freq.write(out_gene_freq[:-1])
file_EGFP.write(out_EGFP[:-1])


file_dNdS.close()
file_gene_freq.close()
file_EGFP.close()


for SNP in SNPs:
    line=""
    for el in SNP:
        line=line+str(el)+";"
    line=line[:-1]+"\n"
    
    out_SNPs=out_SNPs+line
    
file_SNPs=open("SNPs.csv","w")
file_SNPs.write(out_SNPs[:-1])
file_SNPs.close()
            
    
            

            
            
    
            
                       

Script 3: 
Draw dN/dS data onto the genome:

In [ ]:
import csv
from Bio import SeqIO,Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram

file=open("dNdS.csv") #open dNdS data generated from the cell above
csv=csv.reader(file, delimiter=";")
dNdS=[]
genelist={}
for line in csv:
    if line[0]=="sample":
        for gene in line[1:]:
            genelist[gene]=0
    else:
        dNdS.append(line)
    
samples_woRep=["wtACVPXXX","wtFOSPXXX","wtGCVPXXX","wtH2OPXXX","YSACVPXXX","YSFOSPXXX","YSGCVPXXX","YSH2OPXXX"] #samples for which to calculate dN/dS ratios
    
record=SeqIO.read("HSV-1_F-BAC.gb","genbank") #open reference file


gd_diagram = GenomeDiagram.Diagram("HSV-1 F strain") #genome diagram
gd_track_for_features = gd_diagram.new_track(1, name="genome", start=0, end=len(record.seq), scale=0, scale_ticks=0,greytrack=False, greytrack_labels=3) #first track, just including the genome
gd_feature_set = gd_track_for_features.new_set()

start_old=0
start_new=0
strand_old="fw"
strand_new="fw"
for feature in record.features:
    if feature.type=="CDS":
        label=False
        if feature.location.strand==1:
            strand_new="fw"
            start_new=int(feature.location.start.position)
        else:
            strand_new="rev"
            start_new=int(feature.location.end.position)
        if (start_new-start_old<1500) and (strand_new==strand_old):
            label=False
        else:
            label=True
            
        start_old=start_new
        strand_old=strand_new
        
        if "U" in feature.qualifiers["gene"][0]:
            color="grey"
        elif "R" in feature.qualifiers["gene"][0]:
            color="silver"
        gd_feature_set.add_feature(feature, color=color, label=label, label_position="middle", label_angle=90, sigil="ARROW", arrowhead_length=0.2, arrowshaft_height=1.0)

for selection in samples_woRep: #checking per selection
    gd_track_for_features = gd_diagram.new_track(3, name=selection, start=0, end=len(record.seq), scale=1, scale_ticks=0,greytrack=True, greytrack_labels=2)
    gd_feature_set = gd_track_for_features.new_set()
    for sample in dNdS:
        if sample[0][:-4]==selection: #sample features selection
            for ratio,gene in zip(sample[1:],genelist):
                if float(ratio)>2.0: #only including genes with dN/dS ratios above 2.0
                    genelist[gene]+=1
    for gene in genelist:
        for feature in record.features:
            if feature.type =="CDS":
                if feature.qualifiers["gene"][0]==gene:
                    if genelist[gene]>=2: #plotting if at least 2 out of 3 replicates show dN/dS ratios above 2.0
                        if "wt" in selection:
                            color="lightcoral"
                        elif "YS" in selection:
                            color="cornflowerblue"
                        label=False
                        if genelist[gene]==3: #labeling gene if 3 out of 3 replicates with dN/dS above 2.0
                            label=True
                        if feature.location.strand==1:
                            angle=0
                            position="start"
                        else:
                            angle=180
                            position="end"
                        gd_feature_set.add_feature(feature, color=color, label=label, label_angle=angle, sigil="ARROW", arrowhead_length=0.2, arrowshaft_height=1.0,label_position=position)
    for gene in genelist:
        genelist[gene]=0

        
gd_track_for_features = gd_diagram.new_track(11, name="legend", start=0, end=len(record.seq), scale=0, scale_ticks=0,greytrack=False, greytrack_labels=3)
gd_feature_set = gd_track_for_features.new_set() #Legendtrack

pink=(SeqFeature(FeatureLocation(30000, 34000),1))
lightblue=(SeqFeature(FeatureLocation(110000, 114000),1))

gd_feature_set.add_feature(pink, color="lightcoral", label=True, name="dN/dS >2 for at least 2 wt populations", label_angle=0, sigil="ARROW", arrowhead_length=0.2, arrowshaft_height=1.0, label_position="end")
gd_feature_set.add_feature(lightblue, color="cornflowerblue", label=True, name="dN/dS >2 for at least 2 YS populations", label_angle=0, sigil="ARROW", arrowhead_length=0.2, arrowshaft_height=1.0, label_position="end")

        
gd_diagram.draw(
    format="linear",
    orientation="landscape",
    pagesize=(25*cm,10*cm),
    fragments=1,
    start=0,
    end=len(record.seq)
)

gd_diagram.write("dNdS2ormore.pdf", "PDF")
                



Script 4:
Principal component analysis:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import csv

df=pd.read_csv("dNdS.csv",header=0,sep=";") #PCA of dN/dS data, different data can be analyzed though



from sklearn.preprocessing import StandardScaler
features = []
for col in df.columns:
    features.append(col)
features=features[1:]
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,["sample"]].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

from sklearn.decomposition import PCA
pca = PCA(n_components=2) #pca with 2 components
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])


targets=[]
for sample in csv.reader(open("dNdS.csv","r"),delimiter=";"):
    if sample[0]!="sample":
        targets.append(sample[0]) #exclude header

finalDf = pd.concat([principalDf, df[['sample']]], axis = 1)
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 12)
ax.set_ylabel('Principal Component 2', fontsize = 12)
#ax.set_title('2 component PCA', fontsize = 20)
finalDf

transper={"P0":0.05,"PV":0.1,"PX":0.25,"PXV":0.45,"PXX":0.65,"PXXV":0.85,"PXXX":1.0} #giving different passages different transperacy values
for target in targets:
    if "GCV" in target:
        marker="^"
    elif "FOS" in target:
        marker="s"
    elif "ACV" in target:
        marker="o"
    elif "H2O" in target:
        marker="h"
        
    if "PXXXRep2" in target:
        label=target[:2]+" "+target[2:target.find("P")]
    else:
        label="_nolegend_"
        
    if "wt" in target:
        color="lightcoral"
    else:
        color="cornflowerblue"
        
    indicesToKeep = finalDf['sample'] == target

    scatter=ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , s = 30
               ,c=color
               ,alpha=transper[target[target.find("P"):target.find("Rep")]]
               ,label=label
               ,marker=marker)

#plt.xlim(-2.5, 2)
#plt.ylim(-2.5,3)
plt.tight_layout()
legend1=ax.legend( loc='best', frameon=False, fontsize=8)
ax.add_artist(legend1)

plt.savefig("PCA_all.pdf")
plt.show()

finalDf.to_csv("PCA.csv")





Script 5:
Calculate allele frequencies over passaging time:

In [ ]:

import csv


SNPs=[]
samples=[]
for SNP in csv.reader(open("SNPs.csv","r"),delimiter=";"):
    if "sample" not in SNP[0]:
        SNPs.append(SNP)
        if "P0" not in SNP[0]:
            sample=SNP[0][:SNP[0].find("P")]+SNP[0][SNP[0].find("R"):]
            if sample not in samples:
                samples.append(sample)
AF=[]

for SNP in SNPs:
    if "P0" not in SNP[0]:
        line=[SNP[0][:SNP[0].find("P")]+SNP[0][SNP[0].find("R"):],SNP[1],SNP[2],SNP[3],SNP[4],SNP[5],SNP[6],"-","-","-","-","-","-","-"]
        if line not in AF:
            AF.append(line)
    elif ("wt" in SNP[0]) or ("YS" in SNP[0]):
        for sample in samples:
            if SNP[0][:2]==sample[:2]:
                line=[sample,SNP[1],SNP[2],SNP[3],SNP[4],SNP[5],SNP[6],"-","-","-","-","-","-","-"]
                if line not in AF:
                    AF.append(line)
    
passage={"P0":7,
         "PV":8,
         "PX":9,
         "PXV":10,
         "PXX":11,
         "PXXV":12,
         "PXXX":13}        
for SNP in AF:
    for snp in SNPs:
        pas=""
        if (SNP[0]==snp[0][:snp[0].find("P")]+snp[0][snp[0].find("R"):]):
            pas=snp[0][snp[0].find("P"):snp[0].find("R")]
            if (SNP[1:6]==snp[1:6]):
                SNP[passage[pas]]=snp[7]
        elif ("P0" in snp[0]) and (SNP[0][:2]==snp[0][:2]):
            pas="P0"
            if (SNP[1:6]==snp[1:6]):
                SNP[passage[pas]]=snp[7]
out_AF="sample;REF;POS;ALT;changetype;gene;aachange;AF P0;AF PV;AF PX;AF PXV;AF PXX;AF PXXV;AF PXXX\n"            
for SNP in AF:
    line=""
    for el in SNP:
        line=line+str(el)+";"
    line=line[:-1]+"\n"
    out_AF=out_AF+line
outfile=open("AF_overtime.csv","w")
outfile.write(out_AF[:-2])
outfile.close()



Script 6: Plot SNPs over time:

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np

fig, axs = plt.subplots(ncols=2, nrows=4, sharex=True, sharey=True, figsize=(12, 8))

mut={"wt":0, "YS":1}
sel={"ACV":0, "FOS":1, "GCV":2, "H2O":3}
SNPs=[]
samples=[]
passage_number=[5,10,15,20,25,30]
for SNP in csv.reader(open("AF_overtime.csv","r"),delimiter=";"):
    if "sample" not in SNP[0]:
        SNPs.append(SNP)
        if SNP[0][:SNP[0].find("R")] not in samples:
            samples.append(SNP[0][:SNP[0].find("R")])


legend=[]
for sample in samples:
    pos=[-0.14]
    c=0
    gs=[]
    Mut=sample[:2]
    Sel=sample[2:]
    
    for SNP in SNPs:
        if (SNP[5] =="UL30") and (SNP[0][:SNP[0].find("R")]==sample):
            if SNP[7]=="-":
                y=[0]
            else:
                y=[float(SNP[7])]
            x=[0]
            for freq, pas in zip(SNP[8:],passage_number):
                x.append(float(pas))
                if freq!="-":
                    y.append(float(freq))
                else:
                    y.append(0.0)
                
            if "FOS" in SNP[0]:
                marker="s"
            elif "ACV" in SNP[0]:
                marker="o"
            elif "H2O" in SNP[0]:
                marker="h"
            elif "GCV" in SNP[0]:
                marker="^"

            if "wt" in SNP[0]:
                color="lightcoral"
                alpha=0.7
            elif "YS" in SNP[0]:
                color="cornflowerblue"
                alpha=0.7
            if sample not in legend:
                legend.append(sample)
                name=sample
            else:
                name="_nolegend_"
                
                
            axs[sel[Sel],mut[Mut]].plot(x,y,linewidth=2,marker=marker,color=color,alpha=alpha, label=name)
            if (SNP[4]=="NONS"):
                if (SNP[5]=="UL30") and (SNP[6]=="Y557N") :
                    name="Y557S"
                else:
                    name=SNP[6]
            else:
                name=SNP[1]+SNP[2]+SNP[3]
                
            
                
            i=0    
            for position in pos:
                
                if abs(position-y[-1])<0.08:
                    i=i+1
            if x[-1]==30:
                pos.append(y[-1])
                if i==0:
                    if y[-1]>0.05:
                        axs[sel[Sel],mut[Mut]].text(x[-1]+2,y[-1],name,fontsize=7)
            if x[-1]==30:
                c+=1
    axs[sel[Sel],mut[Mut]].plot([0,30],[0.05,0.05],linewidth=3,color="grey",alpha=1.0, linestyle="--")
                   
        

    
axs[3,0].set_xlabel('Passage',fontsize=12)
axs[3,1].set_xlabel('Passage',fontsize=12)
axs[0,0].set_ylabel('Allele Frequency',fontsize=12)
axs[1,0].set_ylabel('Allele Frequency',fontsize=12)
axs[2,0].set_ylabel('Allele Frequency',fontsize=12)
axs[3,0].set_ylabel('Allele Frequency',fontsize=12)

fig.legend(frameon=True,ncol=4,bbox_to_anchor=(0.45, 0.45))
fig.suptitle("UL30 Mutations", fontsize=20)
plt.tight_layout()
plt.savefig("UL30_ot.pdf")

plt.show()
    
    


Script 7:
Ploting SNPs on the genome:

In [ ]:
import csv
from Bio import SeqIO,Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram

file=open("SNPs.csv")
csv=csv.reader(file, delimiter=";")
SNPs=[]
samples_woRep=[]
genelist={}
for line in csv:
    if line[0]!="sample":
        SNPs.append(line)
    
samples_woRep=["wtACV","wtFOS","wtGCV","wtH2O","YSACV","YSFOS","YSGCV","YSH2O"]
    
record=SeqIO.read("HSV-1_F-BAC_Pol.gb","genbank")


gd_diagram = GenomeDiagram.Diagram("HSV-1 F strain")
gd_track_for_features = gd_diagram.new_track(1, name="genome", start=0, end=len(record.seq), scale=0, scale_ticks=0,greytrack=False, greytrack_labels=3)
gd_feature_set = gd_track_for_features.new_set()

start_old=0
start_new=0
strand_old="fw"
strand_new="fw"
for feature in record.features:
    if feature.type=="CDS":
        label=False
        if feature.location.strand==1:
            strand_new="fw"
            start_new=int(feature.location.start.position)
        else:
            strand_new="rev"
            start_new=int(feature.location.end.position)
        if (start_new-start_old<1500) and (strand_new==strand_old):
            label=False
        else:
            label=True
            
        start_old=start_new
        strand_old=strand_new
        
        shaft=0.2
        if "U" in feature.qualifiers["gene"][0]:
            color="grey"
            shaft=0.2
        elif "R" in feature.qualifiers["gene"][0]:
            color="silver"
        elif "Exo" in feature.qualifiers["gene"][0]:
            color="blue"
            shaft=0.85
            label=True
        else:
            color="orange"
            shaft=0.85
            label=True
        if feature.location.strand==1:
            label_angle=0
        else:
            label_angle=180
            
        if "Exo" in feature.qualifiers["gene"][0]:
            label_angle=90
        
        if "site" in feature.qualifiers["gene"][0]:
            label_angle=135
            
     
        label=True
        gd_feature_set.add_feature(feature, color=color, label=label, label_position="start", label_angle=label_angle, sigil="ARROW", arrowhead_length=0.2, arrowshaft_height=shaft)
    
for selection in samples_woRep:
    gd_track_for_features = gd_diagram.new_track(3, name=selection, start=0, end=len(record.seq), scale=1, scale_ticks=0,greytrack=True, greytrack_labels=4)
    gd_feature_set = gd_track_for_features.new_set()
    features=[]
    for SNP in SNPs:
        if (SNP[0][:SNP[0].find("PXXX")]==selection) and (SNP[5] == "UL23"):
            
            if SNP[4]=="NONS":
                name=str(SNP[6])
                if SNP[6]=="Y557N":
                    name="Y557S"
                else:
                    name=str(SNP[6])
            elif SNP[4]=="SYN":
                name=str(SNP[1])+str(SNP[2])+str(SNP[3])
                
            
                
            mut=SeqFeature(FeatureLocation(int(SNP[2]),int(SNP[2])+1))
            
            if "wt" in SNP[0]:
                color="lightcoral"
            elif "YS" in SNP[0]:
                color="cornflowerblue"
            i=0
            for feature in features:
                if abs(feature.location.start-int(SNP[2]))<80:
                    i=i+1
            if i==0:
                label=True
            else:
                label=False
                
        
            
                   
            start_old=start_new
            gd_feature_set.add_feature(mut,name=name,label=label, label_angel=90, sigil="BOX",color=color)
            features.append(mut)

                           
gd_diagram.draw(
    format="linear",
    orientation="landscape",
    pagesize=(25*cm,8*cm),
    fragments=1,
    start=46589,
    end=47719
)

gd_diagram.write("UL23_genome.pdf", "PDF")
                




Script 8:
Ploting coverage data:

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np

fig, axs = plt.subplots(ncols=2, nrows=4, sharex=True, sharey=True, figsize=(12, 8))
mut={"wt":0, "YS":1}
sel={"ACV":0, "FOS":1, "GCV":2, "H2O":3}
csv_file=csv.reader(open("samples_min50DP.csv"), delimiter=";") #table with samples and where to find them
samples_folder=[]
legend=[]
for sample in csv_file: #loading samplelist
    samples_folder.append(sample)
    
for sample in samples_folder:
    if "PXXX" in sample[0]:
        Mut=sample[0][:2]
        Sel=sample[0][2:sample[0].find("P")]
        cov_file=open(sample[1]+"/coverage/"+sample[0]+"_cov_av.txt")
        av=0.0
        cov=[]
        for site in cov_file:
            av=av+int(site.split()[2])
            cov.append(site.split())
        av=av/len(cov)
        x=[]
        y=[]
        for site in cov:
            if (int(site[1])>=46589) and (int(site[1])<=47719):
                x.append(int(site[1]))
                y.append(float(site[2])/av)
        if "FOS" in sample[0]:
            marker="s"
        elif "ACV" in sample[0]:
            marker="o"
        elif "H2O" in sample[0]:
            marker="h"
        elif "GCV" in sample[0]:
            marker="^"

        if "wt" in sample[0]:
            color="lightcoral"
            alpha=0.7
        elif "YS" in sample[0]:
            color="cornflowerblue"
            alpha=0.7

        if "Rep" not in sample[0]:
            sample[0]=sample[0]+"Rep1"
        
        if sample[0][:sample[0].find("P")] not in legend:
            legend.append(sample[0][:sample[0].find("P")])
            name=sample[0][:sample[0].find("P")]
        else:
            name="_nolegend_"
                
                
        axs[sel[Sel],mut[Mut]].plot(x,y,linewidth=2,marker=marker,color=color,alpha=alpha, label=name)
        
axs[3,0].set_xlabel('Position',fontsize=12)
axs[3,1].set_xlabel('Position',fontsize=12)
axs[0,0].set_ylabel('Coverage',fontsize=12)
axs[1,0].set_ylabel('Coverage',fontsize=12)
axs[2,0].set_ylabel('Coverage',fontsize=12)
axs[3,0].set_ylabel('Coverage',fontsize=12)
fig.legend(frameon=True,ncol=4,bbox_to_anchor=(0.45, 0.82))
fig.suptitle("UL23 Coverage", fontsize=20)
plt.tight_layout()
plt.savefig("UL23Cov.pdf")
plt.show()


Script 9:
Calculating and ploting SNP counts:

In [ ]:
import matplotlib.pyplot as plt
import csv


SNPs=[]
samples=[]
samples2=[]
passage_number=[0,5,10,15,20,25,30]

fig = plt.figure(figsize = (6,3.7))

for SNP in csv.reader(open("SNPs.csv","r"),delimiter=";"):
    if "sample" not in SNP[0]:
        SNPs.append(SNP)
        if SNP[0] not in samples:
            samples.append(SNP[0])
        if "P0" not in SNP[0]:
            sample=SNP[0][:SNP[0].find("P")]+SNP[0][SNP[0].find("R"):]
            if sample not in samples2:
                samples2.append(sample)
AF=[]
for SNP in SNPs:
        if "P0" not in SNP[0]:
            line=[SNP[0][:SNP[0].find("P")]+SNP[0][SNP[0].find("R"):],"-","-","-","-","-","-","-"]
            if line not in AF:
                AF.append(line)
        elif ("wt" in SNP[0]) or ("YS" in SNP[0]):
            for sample in samples2:
                if SNP[0][:2]==sample[:2]:
                    line=[sample,"-","-","-","-","-","-","-"]
                    if line not in AF:
                        AF.append(line)
            
passage={"P0":1,
         "PV":2,
         "PX":3,
         "PXV":4,
         "PXX":5,
         "PXXV":6,
         "PXXX":7}              

for sample in samples:
    SNPcount=0
    for SNP in SNPs:
        if SNP[0]==sample:
            SNPcount+=1  
    for SNP in AF:
        pas=""
        if (SNP[0]==sample[:sample.find("P")]+sample[sample.find("R"):]):
            pas=sample[sample.find("P"):sample.find("R")]
            SNP[passage[pas]]=SNPcount
        elif ("P0" in sample) and (SNP[0][:2]==sample[:2]):
            pas="P0"
            SNP[passage[pas]]=SNPcount
out_AF="sample;AF P0;AF PV;AF PX;AF PXV;AF PXX;AF PXXV;AF PXXX\n"            
for SNP in AF:
    line=""
    for el in SNP:
        line=line+str(el)+";"
    line=line[:-1]+"\n"
    out_AF=out_AF+line
outfile=open("SNPcount_overtime.csv","w")
outfile.write(out_AF[:-1])
outfile.close()

for sample in AF:
    if sample[0]!="sample":
        if "Rep2" in sample[0]:
            label=sample[0][:sample[0].find("R")]
        else:
            label="_nolegend_"
            
        x=[]
        y=[]
        for freq, pas in zip(sample[1:],passage_number):
            if freq!="-":
                x.append(float(pas))
                y.append(float(freq))
        if "FOS" in sample[0]:
            marker="s"
        elif "ACV" in sample[0]:
            marker="o"
        elif "H2O" in sample[0]:
            marker="h"
        elif "GCV" in sample[0]:
            marker="^"

        if "wt" in sample[0]:
            color="lightcoral"
            alpha=0.7
        elif "YS" in sample[0]:
            color="cornflowerblue"
            alpha=0.7

        plt.plot(x,y,linewidth=2,marker=marker,color=color,label=label,alpha=alpha)


plt.xlabel("Passage",fontsize=12)
plt.ylabel("SNP count",fontsize=12)
plt.legend(frameon=False, ncol=2)
plt.xlim(0,30.5)
plt.tight_layout()
plt.savefig("SNP_count.pdf")
plt.show()

Script 10:
Generating clone genbank files + multiple FASTA files:

In [ ]:
import csv
from matplotlib import pyplot as plt
import numpy as np
import os
from vcftogenbankDef import vcftogenbank #if in different script


depth=csv.reader(open("coverage/depth_av.csv","r"),delimiter=";") #coverage/depth_av.csv file includes all samplenames + coverage data
samplelist=[]
coveragedic={}
fastadic={}
for el in depth:
    if "name" not in el[0]:
        samplelist.append(el[0])
        coveragedic[el[0]]=float(el[1])

for sample in samplelist:
    if coveragedic[sample]>30.0:
        fastadic[sample]=vcftogenbank("HSV-1_F-BAC.gb",sample) #see script 1 in this notebook, check if samplenames are comparable
wtGCV=""
YSGCV=""
wtH2O=""
YSH2O=""
for sample in fastadic:
    if "wtGCV" in sample:
        wtGCV=wtGCV+">"+sample+"\n"+fastadic[sample]+"\n"
    elif "YSGCV" in sample:
        YSGCV=YSGCV+">"+sample+"\n"+fastadic[sample]+"\n" 
    elif "wtH2O" in sample:
        wtH2O=wtH2O+">"+sample+"\n"+fastadic[sample]+"\n"
    elif "YSH2O" in sample:
        YSH2O=YSH2O+">"+sample+"\n"+fastadic[sample]+"\n" 

wtGCVfile=open("wtGCV.fasta","w")
YSGCVfile=open("YSGCV.fasta","w")
wtH2Ofile=open("wtH2O.fasta","w")
YSH2Ofile=open("YSH2O.fasta","w")

wtGCVfile.write(wtGCV)
YSGCVfile.write(YSGCV)
wtH2Ofile.write(wtH2O)
YSH2Ofile.write(YSH2O)

wtGCVfile.close()
YSGCVfile.close()
wtH2Ofile.close()
YSH2Ofile.close()

        

Script 11:
Heterozygosity Calculation:

In [ ]:
import csv
import vcf
import os
from matplotlib import pyplot as plt
import numpy as np
from Bio import SeqIO,Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation




record=SeqIO.read("HSV-1_F-BAC.gb","genbank") #annotated reference genome

gene_list={}
out="population;"
for feature in record.features:
        if feature.type=="CDS":
            if "R" not in feature.qualifiers["gene"][0]:
                gene_list[feature.qualifiers["gene"][0]]={}
                out=out+feature.qualifiers["gene"][0]+";"
                
                
out=out[:-1]+"\n"


        
            
population=("wtGCV","wtH2O","YSGCV","YSH2O")
clone=("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20")

for pop in population:
    out=out+pop+";"
    for feature in record.features:
        if feature.type=="CDS":
            if "R" not in feature.qualifiers["gene"][0]:
                gene_list[feature.qualifiers["gene"][0]]={}
                
    
    for c in clone:
            if os.path.exists("Run300622/fasta_gb/"+pop+"c"+c+".gb"):
                gb=SeqIO.read("Run300622/fasta_gb/"+pop+"c"+c+".gb","genbank") #open respective genbank file
                
                
                for feature in gb.features:
                    if feature.type=="CDS":
                        if "R" not in feature.qualifiers["gene"][0]:
                            gene=feature.qualifiers["gene"][0]
                            allele=gb.seq[feature.location.start-1:feature.location.end]
                            if allele in gene_list[gene]:
                                gene_list[gene][allele]+=1
                            else:
                                gene_list[gene][allele]=1
    hetero={}
    for gene in gene_list:
        pop_size=0
        hetero[gene]=1
        for allele in gene_list[gene]:
            pop_size+=gene_list[gene][allele]
        for allele in gene_list[gene]:
            gene_list[gene][allele]=float(gene_list[gene][allele]/pop_size)
            hetero[gene]-=np.square(gene_list[gene][allele])
        out=out+str(hetero[gene])+";"
    out=out[:-1]+"\n"
    
out=out[:-1]   
#print(out)
outfile=open("heterozygosity.csv","w")
outfile.write(out)

outfile.close()
        
            


